In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf 
from PIL import Image
import numpy as np
import sys
import os
import csv
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, LeakyReLU, Dense, Dropout
import matplotlib.pyplot as plt 
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util
from keras.callbacks import CSVLogger,ModelCheckpoint,EarlyStopping
from keras.callbacks import ReduceLROnPlateau
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
import os


The google drive is mounted on the google collab drive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_path = "/content/drive/MyDrive/Semester 2 files/EE258/Project EE258 - Dataset/train"
test_path = "/content/drive/MyDrive/Semester 2 files/EE258/Project EE258 - Dataset/test"

**In the drive the training data and the test data are uploaded,and the test path and train path are created seperately.**

In [4]:
x = plt.imread("/content/drive/MyDrive/Semester 2 files/EE258/Project EE258 - Dataset/test/happy/PrivateTest_1140198.jpg")
x.shape

(48, 48)

In [5]:
img_rows,img_cols = 48,48
batch_size =32

The image is grayscale and size of a testing data image is observed.

In [6]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)

It is observed that 28709 images are found in the train data set and 7178 images are found in the test data set.

In [7]:
train = datagen.flow_from_directory(train_path,target_size=(48,48),class_mode="sparse", seed=1, color_mode="grayscale", batch_size=128)
test = datagen.flow_from_directory(test_path,target_size=(48,48),class_mode="sparse", seed=1, color_mode="grayscale", batch_size=128)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [8]:
Classes = 7

We have two train and test data sets

In [9]:
!ls "/content/drive/MyDrive/Face image classification" 

ls: cannot access '/content/drive/MyDrive/Face image classification': No such file or directory


The images are classified as Angry, Disgusted, Fearful, Happy, Neutral, Sad, Surprised as mentioned in the output below.

In [10]:
print(train.class_indices)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised "}

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


The size of the training 

In [11]:
x, y = train.next()
print(x.shape, y.shape)
train.reset()

(128, 48, 48, 1) (128,)


The images are classified as 0-6 types as mentioned in the output below.

In [12]:
class_map = { 
    'angry': 0,
    'disgust' : 1,
    'fear': 2,
    'happy': 3,
    'neutral': 4,
    'sad': 5,
    'surprise': 6
}

The module sequential is created and the module summary is shown in the output.

In [13]:
Emotion_model = Sequential([
                 Conv2D(32, (5,5), input_shape=(48,48,1),padding="same"),
                 LeakyReLU(),
                 BatchNormalization(),
                 Conv2D(32, (5,5), input_shape=(48,48,1),padding="same"),
                 LeakyReLU(),
                 BatchNormalization(),
                 MaxPooling2D((2,2)),
                 Dropout(0.2),           

                 Conv2D(64, (3,3), input_shape=(48,48,1),padding="same"),
                 LeakyReLU(),
                 BatchNormalization(),
                 Conv2D(64, (3,3), input_shape=(48,48,1),padding="same"),
                 LeakyReLU(),
                 BatchNormalization(),
                 MaxPooling2D((2,2)),
                 Dropout(0.2),
                 Conv2D(128, (3,3), input_shape=(48,48,1),padding="same"),
                 LeakyReLU(),
                 BatchNormalization(),
                 Conv2D(128, (3,3), input_shape=(48,48,1),padding="same"),
                 LeakyReLU(),
                 BatchNormalization(),
                 MaxPooling2D((2,2)),
                 Dropout(0.2),
                 Conv2D(256, (3,3), input_shape=(48,48,1),padding="same"),
                 LeakyReLU(),
                 BatchNormalization(),
                 Conv2D(56, (3,3), input_shape=(48,48,1),padding="same"),
                 LeakyReLU(),
                 BatchNormalization(),
                 MaxPooling2D((2,2)),
                 Dropout(0.2),

                 Flatten(),
                 Dense(64, activation='relu'),
                 LeakyReLU(),
                 BatchNormalization(),
                 Dropout(0.5),
                 Dense(64, activation='relu'),
                 LeakyReLU(),
                 BatchNormalization(),
                 Dropout(0.5),
    
                 Dense(len(train.class_indices), activation="softmax")
])
Emotion_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        832       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 48, 48, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        25632     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 48, 48, 32)       128       
 hNormalization)                                        

In [14]:
train_datagen = ImageDataGenerator(
                   rescale = 1./255,
                   rotation_range =30,
                   shear_range = 0.3,
                   zoom_range = 0.3,
                   width_shift_range = 0.4,
                   height_shift_range = 0.4,
                   horizontal_flip = True,
                   fill_mode = 'nearest'
                  )

In [15]:
Emotion_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.00005,
    patience=11,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=7,
    min_lr=1e-7,
    verbose=1,
)

callbacks = [
    early_stopping,
    lr_scheduler,
]

In [17]:
Emotion_model.fit(train, validation_data=test, epochs=10, callbacks=callbacks)

Epoch 1/10
115/225 [==============>...............] - ETA: 38:54 - loss: 2.9591 - accuracy: 0.1596

KeyboardInterrupt: ignored

Data augmentation image rotation 

In [ ]:
def random_rotation(image_array: ndarray):
    # pick a random degree of rotation between 25% on the left and 25% on the right
    random_degree = random.uniform(-25, 25)
    return sk.transform.rotate(image_array, random_degree)


Data augmentation adding noise

In [ ]:
def random_noise(image_array: ndarray):
    # add random noise to the image
    return sk.util.random_noise(image_array)

Data Augmentation flipping the images

In [ ]:
def horizontal_flip(image_array: ndarray):
    # horizontal flip doesn't need skimage, it's easy as flipping the image array of pixels !
    return image_array[:, ::-1]

Transformation for the images,generate a few number of images which could be later used to train a deep network

In [ ]:
# dictionary of the transformations functions we defined earlier
available_transformations = {
    'rotate': random_rotation,
    'noise': random_noise,
    'horizontal_flip': horizontal_flip
}

# random num of transformations to apply
num_transformations_to_apply = random.randint(1, len(available_transformations))

num_transformations = 0
transformed_image = None
while num_transformations <= num_transformations_to_apply:
    # choose a random transformation to apply for a single image
    key = random.choice(list(available_transformations))
    